In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
iris = datasets.load_iris()
import random
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.models_fn.imputer_models import measure_val_error,XGBImputer,MySimpleImputer
from src.data_fn.data_process import simulate_missing_values,replace_infs,test_input_data,infer_cols
from pyTest.test_suite import get_test_data
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import mean_squared_error,f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import SimpleImputer

Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\models_fn\imputer_models.py
Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\data_fn\data_process.py


In [2]:
data = pd.read_csv('..\data\processed\iris_original.csv')
curr_df = simulate_missing_values(data,output_name=None,prop=0.7)
inf_df = curr_df.copy()
inf_df['target'] = inf_df['target'].map({0.0:'A',1.0:'B',2.0:'C'})
inf_df.iloc[0,2] = -np.inf
dtypes = ["numeric", "numeric", "numeric", "numeric", "categorical"]
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
imp1 = MySimpleImputer(dtype_list=dtype_list)


In [3]:
data = pd.read_csv(r'..\data\processed\adult_nans_02_2k.csv')
# data.loc[1:2000,:].to_csv(r'..\data\processed\adult_nans_02.csv',index=False)
dtypes = ["numeric", "categorical", "numeric", "categorical", "numeric", "categorical", "categorical", "categorical", "categorical", "categorical",'numeric','numeric','numeric','categorical','categorical']
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
data

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,NaN,White,Male,0.0,NaN,40.0,United-States,<=50K
1,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,NaN
2,NaN,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,NaN,NaN,40.0,Cuba,<=50K
3,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,NaN,NaN,NaN,NaN,NaN,0.0,40.0,United-States,<=50K
4,49.0,Private,NaN,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,NaN,0.0,16.0,Jamaica,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NaN,Self-emp-not-inc,30599.0,Masters,14.0,Married-spouse-absent,Prof-specialty,NaN,White,Female,0.0,0.0,50.0,United-States,<=50K
1996,37.0,Private,379522.0,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,55.0,United-States,>50K
1997,51.0,NaN,NaN,HS-grad,9.0,NaN,Craft-repair,Unmarried,NaN,Male,0.0,0.0,38.0,United-States,NaN
1998,35.0,Private,82552.0,NaN,NaN,Never-married,NaN,Unmarried,White,Male,0.0,0.0,35.0,United-States,NaN


In [4]:
# # imp1 = MySimpleImputer(dtype_list=dtype_list)
# imp1 = XGBImputer(dtype_list=dtype_list)
imp1 = MySimpleImputer(dtype_list=dtype_list)
res = imp1.impute(data)
res


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Husband,White,Male,0.0,91.582543,40.0,United-States,<=50K
1,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
2,38.855466,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,999.982901,91.582543,40.0,Cuba,<=50K
3,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Craft-repair,Husband,White,Male,999.982901,0.0,40.0,United-States,<=50K
4,49.0,Private,193085.347442,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,999.982901,0.0,16.0,Jamaica,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38.855466,Self-emp-not-inc,30599.0,Masters,14.0,Married-spouse-absent,Prof-specialty,Husband,White,Female,0.0,0.0,50.0,United-States,<=50K
1996,37.0,Private,379522.0,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,55.0,United-States,>50K
1997,51.0,Private,193085.347442,HS-grad,9.0,Married-civ-spouse,Craft-repair,Unmarried,White,Male,0.0,0.0,38.0,United-States,<=50K
1998,35.0,Private,82552.0,HS-grad,10.150531,Never-married,Craft-repair,Unmarried,White,Male,0.0,0.0,35.0,United-States,<=50K


In [5]:
measure_val_error(data,imp1)

{'39': 7.58,
 ' State-gov': 0.88,
 ' 77516': 51652.54,
 ' Bachelors': 0.69,
 ' 13': 1.62,
 ' Never-married': 0.76,
 ' Adm-clerical': 0.66,
 ' Not-in-family': 0.73,
 ' White': 0.96,
 ' Male': 0.91,
 ' 2174': 1410.83,
 ' 0': 321.78,
 ' 40': 8.86,
 ' United-States': 0.97,
 ' <=50K': 0.91}

In [6]:
test_df = data.copy()
test_df = replace_infs(test_df)
test_df = test_df.dropna(axis=0, how="any")
test_df

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
15,32.0,Private,186824.0,HS-grad,9.0,Never-married,Machine-op-inspct,Unmarried,White,Male,0.0,0.0,40.0,United-States,<=50K
24,19.0,Private,168294.0,HS-grad,9.0,Never-married,Craft-repair,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K
189,24.0,Private,159567.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
191,47.0,Private,120939.0,Some-college,10.0,Married-civ-spouse,Tech-support,Husband,White,Male,0.0,0.0,45.0,United-States,<=50K
221,54.0,Private,176681.0,HS-grad,9.0,Married-civ-spouse,Adm-clerical,Husband,Black,Male,0.0,0.0,20.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849,63.0,?,231777.0,Masters,14.0,Married-civ-spouse,?,Husband,White,Male,0.0,0.0,30.0,United-States,<=50K
1918,32.0,Private,120426.0,HS-grad,9.0,Separated,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
1934,38.0,Local-gov,255454.0,Bachelors,13.0,Separated,Prof-specialty,Unmarried,Black,Male,0.0,0.0,40.0,United-States,<=50K
1960,50.0,Private,147629.0,Masters,14.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024.0,0.0,45.0,United-States,>50K


In [7]:
test_input_data(data)

{'is_empty': False,
 'dtypes': {dtype('float64'), dtype('O')},
 'prop_missing': 0.20053333333333334,
 'n_full_nan_rows': 0,
 'n_full_nan_cols': 0,
 'bool_all_rows_have_nans': False}

In [8]:
# data[' 2174'].nunique()/ data[' 2174'].count() < 0.05


# # # # 1.*data[' 2174'].value_counts(normalize=True)
# low_cardinality_cols = [col for col in data.columns if data[col].nunique()/ data[col].count() < 0.02]
# low_cardinality_cols

In [8]:
# data[.value_counts()
infer_cols(data)
# len(np.unique(data[' 2174']))

([' State-gov',
  ' Bachelors',
  ' Never-married',
  ' Adm-clerical',
  ' Not-in-family',
  ' White',
  ' Male',
  ' United-States',
  ' <=50K'],
 {' <=50K',
  ' Adm-clerical',
  ' Bachelors',
  ' Male',
  ' Never-married',
  ' Not-in-family',
  ' State-gov',
  ' United-States',
  ' White'})